In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import pycountry
import geopandas as gpd
import requests

import altair as alt
from vega_datasets import data
import eco_style
alt.themes.enable('colab_site')
import json

In [14]:
json.dumps(eco_style.colab_site())

'{"config": {"font": "Roboto", "mark": {"line": {"interpolate": "linear"}}, "view": {"stroke": "transparent", "width": 400, "height": 300}, "range": {"category": ["#179fdb", "#e6224b", "#f4c245", "#122b39", "#eb5c2e", "#36b7b4"], "diverging": ["#E6224B", "#E54753", "#C9C9C9", "#179FDB", "#122B39"], "heatmap": ["#C9C9C9", "#179FDB", "#0063AF", "#122B39"], "ordinal": ["#00A767", "#36B7B4", "#179FDB", "#0063AF", "#243B5A"]}, "axisX": {"labelColor": "#122b39", "tickColor": "#122b39", "domainColor": "#122b39", "domainOpacity": 0.5, "gridOpacity": 0, "labelFont": "Roboto", "labelAngle": 0, "labelAlign": "center", "labelFontSize": 11, "labelPadding": 5, "tickCount": 10, "tickSize": 0, "title": ""}, "axisY": {"labelColor": "#122b39", "labelFont": "Roboto", "tickColor": "#122b39", "domainColor": "#122b39", "gridColor": "#122b39", "gridDash": [1, 5], "gridOpacity": 0.5, "labelPadding": 5, "labelFontSize": 11, "domainOpacity": 0.5, "tickSize": 0, "title": null, "titleAlign": "left", "titleAngle":

In [12]:
df = pd.read_csv("data/tza.csv")
df = df.melt(id_vars=["Manufacturing VA/GDP"], var_name="date", value_name="value")
df.columns = ['country', 'date', 'value']
df = df.dropna()

# add a label to the final row for each country
df['label'] = np.where(df['date'] == df.groupby('country')['date'].transform('max'), df['country'], "")

base = alt.Chart(df).encode(
    x=alt.X('date:T', axis=alt.Axis(title='')),
    y=alt.Y('value:Q', axis=alt.Axis(title='', format='%')),
    color=alt.Color("country:N", scale={
    "domain": [ 'Tanzania',  'Uganda', 'South Africa', 'Kenya', 'Ethiopia',
       'Cambodia', 'Ghana'],
    "range": ["#001f3f", "#4269d0", "#efb118", "#ff725c", "#6cc5b0", "#3ca951", "#ff8ab7", "#a463f2", "#97bbf5", "#9c6b4e"]
}, legend=None)

)


line = base.mark_line().encode(
    strokeDash=alt.condition(alt.datum.country == "Tanzania", alt.value([0,0]), alt.value([5,2])),
    size=alt.condition(alt.datum.country == "Tanzania", alt.value(2), alt.value(1.5)),
)


dx = {
    "Tanzania": 0,
    "Uganda": 0,
    "South Africa": -5,
    "Kenya": -5,
    "Ethiopia": 0,
    "Cambodia": 0,
    "Ghana": 0,
}

labels = base.mark_text(
    align='left',
    dx=5,
    dy=alt.expr(f"{json.dumps(dx)}[datum.country]")
).encode(
    text=alt.Text('label:N'),
    size=alt.condition(alt.datum.country == "Tanzania", alt.value(12), alt.value(10)),
)


chart = (line + labels).properties(
    width="container",
    height=300,
    title=alt.TitleParams(
        text="Manufacturing value added",
        subtitle=["Percentage of GDP", "Source: World Bank, author's calculations"],
        anchor="start",
        dx=27,
        fontSize=14,
        subtitleFontSize=12,
        fontWeight="normal",
        subtitleFontWeight="lighter",
    
    )
)

chart.save("charts/tza.json")
chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

In [18]:
df.country.unique()

array(['Uganda', 'South Africa', 'Kenya', 'Tanzania', 'Ethiopia',
       'Cambodia', 'Ghana'], dtype=object)

In [13]:
df.drop_duplicates(subset=['country'], keep='last').sort_values('value', ascending=False).head(10)

,country,date,value
231,Cambodia,2023,0.187300
232,Uganda,2023,0.159609
234,South Africa,2023,0.113928
233,Ghana,2023,0.110411
228,Kenya,2022,0.088481
236,Tanzania,2023,0.084998
237,Ethiopia,2023,0.056382
